# Goldman Sachs Marquee Security Master

## Introduction
This notebook provides examples using the GS-Quant Security Master SDK.  The examples  will help users resolve current identifiers to historical identifiers, retrieve detailed security data, and perform various queries. <br><br>
**Security Master developer documentation is available here:**
[https://developer.gs.com/p/docs/services/data/securities-master/](https://developer.gs.com/p/docs/services/data/securities-master/)<br><br>
**GS-Quant SDK documentation is available here:**
[https://developer.gs.com/docs/gsquant/data/accessing-data/secmaster_sdk/](https://developer.gs.com/docs/gsquant/data/accessing-data/secmaster_sdk/)<br><br>

In [ ]:
import pandas as pd
import datetime as dt

from IPython.display import Markdown
from pprint import pprint
from gs_quant.target.secmaster import SecMasterAssetType
from gs_quant.api.gs.secmaster import GsSecurityMasterApi
from gs_quant.markets.securities import SecurityIdentifier
from gs_quant.session import GsSession

GsSession.use(client_id="INSERT HERE", client_secret="INSERT HERE")

## Sec Master ID Sorter
In the following example is a function using the Get Many Securities method where the user can provide any identifier and have them resolved to an immutable Sec Master ID ('id') along with other market standard symbology.

In [ ]:
def sec_master_id_sorter(identifiers):
    """
    Fetch security data and normalize the identifiers.

    :param identifiers: List of security identifiers
    :return: DataFrame containing normalized identifiers
    """
    data = GsSecurityMasterApi.get_many_securities(identifier=identifiers, isPrimary=True)
    results = pd.DataFrame(data['results'])
    return pd.json_normalize(results['identifiers'])


# Example usage
identifiers = ["GOOGL UW", "VOD.N", "US67066G1040"]
normalized_identifiers = sec_master_id_sorter(identifiers)
pd.DataFrame(normalized_identifiers)

## Map Current Symbology to Historical Symbology
The following example is a function using the Get Many Securities method that resolves current symbology to historical symbology, ensuring that the user has correct identifiers as of a given point in time.

In [ ]:
def get_historical_ids(identifiers, date, listed=True, id_type='bbid'):
    """
    Fetch historical identifiers for multiple securities.

    :param identifiers: List of current identifiers of the securities.
    :param date: The date for which the historical identifiers are required.
    :param listed: Whether to filter by listed securities.
    :param id_type: The type of identifier to retrieve.
    :return: Dictionary mapping each identifier to its historical identifier.
    """
    historical_ids = {}
    for identifier in identifiers:
        try:
            data = GsSecurityMasterApi.get_many_securities(identifier=[identifier], isPrimary=True)
            if not data.get('results'):
                raise ValueError(f"No security found for identifier {identifier}")

            secm_id = data['results'][0]['id']
            historical_data = GsSecurityMasterApi.get_identifiers(secmaster_id=secm_id)

            match = next(
                (
                    record['value']
                    for record in historical_data
                    if dt.datetime.strptime(record['startDate'], '%Y-%m-%d').date()
                    <= date
                    <= (
                        dt.datetime.strptime(record['endDate'], '%Y-%m-%d').date()
                        if record['endDate'] != '9999-99-99'
                        else dt.date.today()
                    )
                    and (not listed or record['type'] == id_type)
                ),
                None,
            )
            historical_ids[identifier] = match or identifier
        except Exception as e:
            print(f"Error fetching historical ID for {identifier}: {e}")
            historical_ids[identifier] = None
    return historical_ids


# Example usage
identifiers = ["META UW", "GOOG UW", "XYZ UN"]
query_date = dt.date(2017, 1, 1)

try:
    historical_ids = get_historical_ids(identifiers, query_date)
    for identifier, historical_id in historical_ids.items():
        print(f"Historical ID for {identifier} as of {query_date}: {historical_id}")
except ValueError as e:
    print(f"Error: {e}")

## Query by Identifier
This example allows the user to quickly retrieve the latest available information about a specific security.

In [ ]:
data = GsSecurityMasterApi.get_security_data(id_value="GS UN", id_type=SecurityIdentifier.BBID)
pprint(data)

## Query by Attribute
This example allows the user to specify an asset type or set of identifiers to retrieve matching securities.

In [ ]:
data = GsSecurityMasterApi.get_many_securities(type_=SecMasterAssetType.Common_Stock, limit=1)
results = data['results']
pprint(results)

## Get All Securities
This example is an expansive layer over 'Query by Attribute' and is tailored for retrieving an extensive list of assets within a specific type.
It simplifies the user experience by automatically handling the scrolling.
The output below shows the first two results for Dutch Certificates.  This example was selected to optimize speed since there are comparably few assets of this type.

In [ ]:
data = GsSecurityMasterApi.get_all_securities(type_=SecMasterAssetType.Dutch_Cert)
pprint(data['results'][0:2])

## Get Identifier History
This example provides a chronological account of how a particular identifier has changed for a security over time.
This method only accepts 'id' as an input, which is the immutable Sec Master ID.

In [ ]:
secm_id = "GSPD227284E459"
pd.DataFrame(GsSecurityMasterApi.get_identifiers(secmaster_id=secm_id))

## Map Securities
This example converts between different types of security identifiers and accounts for historical changes.

In [ ]:
pd.DataFrame(
    GsSecurityMasterApi.map(
        SecurityIdentifier.GSID,
        ["40602012"],
        start_date="2007-01-01",
        end_date="2024-10-29",
        output_types=[SecurityIdentifier.BBID, SecurityIdentifier.CUSIP, SecurityIdentifier.SEDOL],
    )
)

## Search Securities
This example allows for full-text searches across many fields, including identifiers and company names.

In [ ]:
results = GsSecurityMasterApi.search(
    q="TESLA", is_primary=True, active_listing=True, type_=SecMasterAssetType.Common_Stock
)
[
    f"The bbid is: {data['identifiers']['bbid']} and the issuer name is: {data['issuer']['name']}  and the type is: {data['type']}."
    for data in results
]

## Deltas - Get Recent Changes
This example retrieves the list of recent identifier changes within a specified time range.

In [ ]:
start_time = dt.datetime.now() - dt.timedelta(hours=1)
data = GsSecurityMasterApi.get_deltas(raw=False, start_time=start_time)
pd.DataFrame(data['results'])

## Issuers - Capital Structure
This example provides all listed securities types for a given issuer. 

In [ ]:
bbid = "META UW"
data = GsSecurityMasterApi.get_capital_structure(id_value=bbid, id_type=SecurityIdentifier.BBID)
res = data['results'][0]
display(Markdown(f"Capital structure for {bbid} matched {res['issuerName']}"))
display(pd.DataFrame([data['assetTypesTotal']]))
for t in res['types']:
    display(Markdown(f"**{t}**"))
    display(pd.DataFrame(res['types'][t][:5]))

## Exchanges
This example retrieves detailed information about exchanges.

In [ ]:
data = GsSecurityMasterApi.get_exchanges()
pd.DataFrame(data['results'][:5])

### Exchanges - Query by Exchange Code
This example retrieves detailed information about exchanges by selected exchange codes.

In [ ]:
data = GsSecurityMasterApi.get_exchanges(mic="XBUE")
pd.DataFrame(data['results'])

### Exchanges - Get Exchange Code History
This example provides the history of exchange codes.
This method only accepts gsExchangeId as an input.

In [ ]:
results = GsSecurityMasterApi.get_exchange_identifiers_history(gs_exchange_id=2)
pd.DataFrame(results)

## Listed Derivatives by Underlyer
This example provides the listed derivatives for a given underlying security.
This method only accepts 'id' as an input, which is the immutable Sec Master ID.

In [ ]:
underlyers = GsSecurityMasterApi.get_securities_by_underlyers(
    id_value="GSPD100E0", limit=5, type_=SecMasterAssetType.Equity_Option
)
print(f"Offset key is: {underlyers.get('offsetKey', 'No offset key returned')}.")
display(pd.DataFrame(underlyers['results']))

if 'offsetKey' in underlyers:
    underlyers = GsSecurityMasterApi.get_securities_by_underlyers(
        id_value="GSPD100E0", limit=5, type_=SecMasterAssetType.Equity_Option, offset_key=underlyers['offsetKey']
    )
    print(f"Offset key is: {underlyers.get('offsetKey', 'No offset key returned')}.")
    display(pd.DataFrame(underlyers['results']))

### Listed Derivatives by Underlyer - fetch all matching securities
This example gets all matching listed derivatives

In [ ]:
underlyers = GsSecurityMasterApi.get_securities_by_underlyers(
    id_value="GSPD100E0", type_=SecMasterAssetType.Equity_Option, scroll_all_pages=True
)
print(f"Total assets count is: {underlyers.get('totalResults', 'No totalResults returned')}.")
display(pd.DataFrame(underlyers['results']))

### Listed Derivatives by Underlyer - Currency and Country Code Filter
This example filters listed derivatives by currency and country code.


In [ ]:
underlyers = GsSecurityMasterApi.get_securities_by_underlyers(
    id_value="GSPD100E0", limit=5, type_=SecMasterAssetType.Equity_Option, currency='USD', country_code='US'
)

display(pd.DataFrame(underlyers['results']))

### Listed Derviatives by Underlyer - Effective Date Filter
This example filters listed derivatives by effective date.

In [ ]:
underlyers = GsSecurityMasterApi.get_securities_by_underlyers(
    id_value="GSPD100E0",
    limit=5,
    type_=SecMasterAssetType.Equity_Option,
    effective_date='2023-01-01',
    country_code='US',
)
display(pd.DataFrame(underlyers['results']))

### Listed Derivatives by Underlyer - Inactive Filter
This example includes inactive listed derivatives in the results.

In [ ]:
underlyers = GsSecurityMasterApi.get_securities_by_underlyers(
    id_value="GSPD100E0", limit=5, type_=SecMasterAssetType.Equity_Option, include_inactive=True, country_code='US'
)
display(pd.DataFrame(underlyers['results']))